![Logo](assets/effocr_logo.png)

## Installation and Setup

Installing EffOCR is easy with `pip`!

In [ ]:
%pip install efficient_ocr

For the remainder of the demo, we'll need import the main `EffOCR` class, as well as a few other helpful libraries.

In [ ]:
from efficient_ocr import EffOCR

In [1]:
import yaml
# import ...

## EffOCR Configs

Whether you're running training or inference with EffOCR, you'll need to speciy a config! Fortunately, EffOCR only requires you directly specify a single config file in `YAML` format. (If you want to use MMDetection as a backend for object detection, you'll also need an MMDetection config file as well.)

Here's how it looks, for example:

In [4]:
with open('./config/config_dummy.yaml') as f:
    print(yaml.safe_dump(yaml.safe_load(f), explicit_start=True))

---
Global:
  char_only: false
  language: en
  recognition_only: false
  wandb_project: effocr_package_test_locca
Line:
  batch_size: 16
  conf_thresh: 0.2
  device: cpu
  epochs: 50
  input_shape: (640, 640)
  iou_thresh: 0.15
  max_det: 200
  min_seg_ratio: 2
  model_backend: yolo
  model_path: ./models/yolo/line_model.pt
  num_cores: null
  providers: null
  training_name: effocr_line
  visualize: null
Localizer:
  batch_size: 16
  conf_thresh: 0.25
  device: cpu
  epochs: 50
  input_shape: (640, 640)
  iou_thresh: 0.1
  language: en
  max_det: 200
  mmdet_config: null
  model_backend: yolo
  model_path: ./models/yolo/localizer_model.pt
  num_cores: null
  onnx_providers: null
  training_name: effocr_localizer
  vertical: false
  visualize: null
Recognizer:
  char:
    adamw_beta1: 0.9
    adamw_beta2: 0.999
    ascender: true
    aug_paired: false
    batch_size: 128
    char_only_sampler: false
    char_trans_version: 2
    dec_lr_factor: 0.9
    default_font_name: Noto
    diff_

## 1. Zero Shot Inference

In [ ]:
### Demonstrate Zero Shot capabilities

### Read Images (find friendly examples)
img = cv2.imread(XXXX)

### Display within notebook


In [ ]:
engine = effocr.EffOCR(config = 'path/to/config.yaml')

In [ ]:
# Run EffOCR
result = engine.infer(img)

# Display results
print(result.text)

In [ ]:
# Show Visualization
img = effocr.display_detections(img, result.preds)
display(image)

## 2. Few Shot Learning

In [ ]:
### Show

## 3. End-to-End Training

In [ ]:
### Show data json, image folder, demonstrate full training